In [ ]:
from modulos import ann, eda, ml_func
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

# ML - Bacia do Rio Negro 2

Usando a metodologia do ML - Bacia do Rio Negro 1 + Cota das estações montantes

### Estações

1. Rio Negro

2. Rio Branco

3. Rio Tiquié + Uaupés

### Correlação cruzada
Aplicar lags nas estações montantes para verificar qual possui maior correlação com a cota atual

### Aprendizado de Máquina

##### Separação dos conjuntos

##### Funções

##### MLP

##### LSTM